In [51]:
from tqdm import tqdm
import json
import random
import torch
import numpy as np
from transformers import AutoTokenizer, AutoConfig, AutoModel
from classifier import MistralForSequenceClassification
import os

# os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [54]:
data = []
texts, labels = [], []
texts_, labels_ = [],[]


with open('sfw-dataset.jsonl') as fopen:
    
    for x in fopen:
        data.append(json.loads(x))

In [55]:
porn = []

with open('porn-sfw-dataset.jsonl') as fopen:
    
    for x in fopen:
        texts.append(json.loads(x)['Content'])
        labels.append(1)

In [56]:


labels_sentiment = ['racist', 'religion insult','psychiatric or mental illness', 'sexist','harassment','informative','safe for work','hate']

for l in data:
    try:
        if eval(l['output'])['label'] != 'porn':
            texts_.append(l['left'])
            labels_.append(0)
        else:
            continue
    except SyntaxError as e:
        if eval(l['output']+ '}')['label'] != 'porn':
            texts_.append(l['left'])
            labels_.append(0)
        elif eval(l['output']+ '}')['label'] == 'porn':
            continue
    except Exception as e: 
        print(e)
        continue
len(texts),len(texts_)

(571, 104889)

In [58]:
random.shuffle(texts_)
sample_non_sexists = random.sample(texts_, len(texts)*2)
label_non_sexists = labels_[:len(sample_non_sexists)]

len(sample_non_sexists)

1142

In [59]:
texts.extend(sample_non_sexists)
labels.extend(label_non_sexists)

In [60]:
len(texts)

1713

In [61]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_Y, test_Y = train_test_split(
    texts, labels, test_size = 0.1
)

In [62]:
np.unique(labels, return_counts = True)

(array([0, 1]), array([1142,  571]))

In [63]:
id2label = {'0':'non porn','1':'porn'}

In [64]:
config = AutoConfig.from_pretrained('mesolitica/malaysian-mistral-191M-MLM-512')
config.num_labels = len(set(labels))
config.vocab = ['non porn', 'porn']

/home/ubuntu/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [65]:
model = MistralForSequenceClassification.from_pretrained('mesolitica/malaysian-mistral-191M-MLM-512', config = config)
_ = model.cuda()

Some weights of MistralForSequenceClassification were not initialized from the model checkpoint at mesolitica/malaysian-mistral-191M-MLM-512 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [66]:
tokenizer = AutoTokenizer.from_pretrained('mesolitica/malaysian-mistral-191M-MLM-512')

In [67]:
trainable_parameters = [param for param in model.parameters() if param.requires_grad]
trainer = torch.optim.AdamW(trainable_parameters, lr = 2e-4)

In [68]:
batch_size = 20
epoch = 100


best_dev_acc = -np.inf
patient = 3
current_patient = 0

for e in range(epoch):
    pbar = tqdm(range(0, len(train_X), batch_size))
    losses = []
    for i in pbar:
        trainer.zero_grad()
        x = train_X[i: i + batch_size]
        y = np.array(train_Y[i: i + batch_size])
        
        padded = tokenizer(x, padding = 'longest', return_tensors = 'pt')

        padded['labels'] = torch.from_numpy(y)
        for k in padded.keys():
            padded[k] = padded[k].cuda()
        
        padded.pop('token_type_ids', None)

        loss, pred = model(**padded, return_dict = False)
        loss.backward()
        
        grad_norm = torch.nn.utils.clip_grad_norm_(trainable_parameters, 5.0)
        trainer.step()
        losses.append(float(loss))
        
    dev_predicted = []
    for i in range(0, len(test_X), batch_size):
        x = test_X[i: i + batch_size]
        y = np.array(test_Y[i: i + batch_size])
        padded = tokenizer(x, padding = 'longest', return_tensors = 'pt')
        padded['labels'] = torch.from_numpy(y)
        for k in padded.keys():
            padded[k] = padded[k].cuda()
        
        padded.pop('token_type_ids', None)

        loss, pred = model(**padded, return_dict = False)
        dev_predicted.append((pred.argmax(axis = 1).detach().cpu().numpy() == y).mean())
        
    dev_predicted = np.mean(dev_predicted)
    
    print(f'epoch: {e}, loss: {np.mean(losses)}, dev_predicted: {dev_predicted}')
    
    if dev_predicted >= best_dev_acc:
        best_dev_acc = dev_predicted
        current_patient = 0
        model.save_pretrained('porn-mistral-mlm')
    else:
        current_patient += 1
    
    if current_patient >= patient:
        break

100%|██████████| 78/78 [00:16<00:00,  4.83it/s]


epoch: 0, loss: 0.22593505177885675, dev_predicted: 0.9462962962962962


100%|██████████| 78/78 [00:16<00:00,  4.85it/s]


epoch: 1, loss: 0.02160757591068082, dev_predicted: 0.9555555555555555


100%|██████████| 78/78 [00:16<00:00,  4.84it/s]


epoch: 2, loss: 0.009253401805101301, dev_predicted: 0.9351851851851851


100%|██████████| 78/78 [00:16<00:00,  4.83it/s]


epoch: 3, loss: 0.0006005912231534566, dev_predicted: 0.9296296296296295


100%|██████████| 78/78 [00:16<00:00,  4.82it/s]


epoch: 4, loss: 5.9047970773043256e-05, dev_predicted: 0.9296296296296295


In [69]:
padded = tokenizer(texts[:10], padding = True, return_tensors = 'pt')
for k in padded.keys():
    padded[k] = padded[k].cuda()

In [70]:
batch_size = 20
epoch = 100

real_Y = []
for i in tqdm(range(0, len(test_X), batch_size)):
    x = test_X[i: i + batch_size]
    y = np.array(test_Y[i: i + batch_size])
    padded = tokenizer(x, padding = 'longest', return_tensors = 'pt')
    padded['labels'] = torch.from_numpy(y)
    for k in padded.keys():
        padded[k] = padded[k].cuda()
    padded.pop('token_type_ids', None)

    loss, pred = model(**padded,return_dict=False)
    real_Y.extend(pred.argmax(axis = 1).detach().cpu().numpy().tolist())

100%|██████████| 9/9 [00:00<00:00, 14.74it/s]


In [71]:
from sklearn import metrics

print(
    metrics.classification_report(
        real_Y, test_Y, target_names = config.vocab,
        digits = 5
    )
)

              precision    recall  f1-score   support

    non porn    0.93023   0.97561   0.95238       123
        porn    0.93023   0.81633   0.86957        49

    accuracy                        0.93023       172
   macro avg    0.93023   0.89597   0.91097       172
weighted avg    0.93023   0.93023   0.92879       172

